In [17]:
%load_ext autoreload
%autoreload 2

import utils
import config
import pandas as pd
import plotly.express as px

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
keywords = config.Keywords.load()
grants = config.Grants.load()
cats = config.Categories.load()

In [30]:
df = grants.explode("linked_organizations").groupby("linked_organizations").size().sort_values(ascending=False)
df[df.index.str.startswith("University of Melb")]

linked_organizations
University of Melbourne                                                                    12491
University of Melbourne Melbourne Medical School                                              96
University of Melbourne Faculty of Science                                                    79
University of Melbourne Graduate School of Education                                          36
University of Melbourne Research                                                              33
University of Melbourne Melbourne School of Engineering                                       30
University of Melbourne at The Doherty Institute                                              21
University of Melbourne Law School                                                            21
University of Melbourne Institute                                                             20
University of Melbourne Faculty of Medicine Dentistry and Health Sciences                     19
Universit

In [21]:
df = pd.DataFrame(sorted_orgs, columns=["Organization", "Grant Count"])

In [24]:
df[df.Organization.str.startswith("University of Melb")]

,Organization,Grant Count
1,University of Melbourne,12491
447,University of Melbourne Melbourne Medical School,96
518,University of Melbourne Faculty of Science,79
1093,University of Melbourne Graduate School of Edu...,36
1185,University of Melbourne Research,33
1293,University of Melbourne Melbourne School of En...,30
1801,University of Melbourne Law School,21
1802,University of Melbourne at The Doherty Institute,21
1892,University of Melbourne Institute,20
2008,University of Melbourne Faculty of Medicine De...,19
